# Bibliotecas necessárias, inclusive para rodar no power bi

In [12]:
# Instalando bibliotecas necessárias

!pip install pandas==1.3.5
!pip install numpy==1.21.6
!pip install matplotlib==3.5.3
!pip install seaborn==0.12.0
!pip install fundamentus==0.2.0
!pip install yfinance==0.1.87

# Dados de cotações

In [10]:
# Código completo da obtenção dos dados históricos das cotações da carteira de 01/08/2022 à 01/08/2023
# Importando a biblioteca
import yfinance as yf

# Definindo a carteira de ações
carteira_yf = ['ABEV3.SA', 'B3SA3.SA', 'ELET3.SA', 'GGBR4.SA', 'ITSA4.SA',
               'PETR4.SA', 'RENT3.SA', 'SUZB3.SA', 'VALE3.SA', 'WEGE3.SA']

# Carregando os dados da carteira
df = yf.download(carteira_yf, start="2022-08-01", end="2023-08-01")

# Passando os ativos para o multindex do df
cotacoes = df.stack(level=1)

# Resetando os índices e renomenado a coluna dos ativos
cotacoes = cotacoes.reset_index().rename(columns={'level_1': 'Ativo'})

# Organizando o df
cotacoes = cotacoes[["Date", "Open", "High", "Low", "Close", "Ativo"]]

del carteira_yf, df

cotacoes

[*********************100%***********************]  10 of 10 completed


,Date,Open,High,Low,Close,Ativo
0,2022-08-01,14.860000,14.890000,14.430000,14.630000,ABEV3.SA
1,2022-08-01,11.020000,11.120000,10.710000,10.870000,B3SA3.SA
2,2022-08-01,45.799999,46.889999,45.660000,46.500000,ELET3.SA
3,2022-08-01,19.547619,19.547619,18.650793,18.785713,GGBR4.SA
4,2022-08-01,7.457636,7.483501,7.285205,7.362799,ITSA4.SA
...,...,...,...,...,...,...
2495,2023-07-31,30.299999,31.150000,30.299999,31.110001,PETR4.SA
2496,2023-07-31,66.888138,67.506729,66.289497,67.067726,RENT3.SA
2497,2023-07-31,47.389999,48.610001,47.340000,48.070000,SUZB3.SA
2498,2023-07-31,68.000000,69.680000,67.839996,69.160004,VALE3.SA


# Gráfico de cotações (Funcionando apenas no power bi - devido ao filtro do ativo)

In [ ]:
# Códido usado para criação do dash de cotações (funcionando apenas no power bi)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

# Configurações iniciais de fonte
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams["font.sans-serif"] = 'Verdana'

def candlestick(date, open, high, low, close):
    fig, ax = plt.subplots(figsize=(30,11), dpi=72, facecolor='#E5E5E5')
    ax.set_facecolor("#E5E5E5")

    # Definindo as cores de cada candle
    cores = ["green" if close > open else "red" for close, open in zip(close, open)]

    # Candlestick corpo + pavio
    sns.barplot(x=date, y=np.abs(open-close), bottom=np.min((open,close), axis=0), width=0.8, palette=cores, ax = ax)
    sns.barplot(x=date, y=high-low, bottom=low, width=0.1, palette=cores, ax = ax)

    # Média móvel de 7 e 30 períodos
    mav_7 = close.rolling(7).mean()
    mav_30 = close.rolling(30).mean()
    sns.lineplot(x=ax.get_xticks(), y=mav_7, label = "MAV 7", color = "#E76F51", linewidth = 4, ax = ax)
    sns.lineplot(x=ax.get_xticks(), y=mav_30, label = "MAV 30", color = "#023E8A", linewidth = 4, ax = ax)
    plt.legend(fontsize=28, facecolor="#E5E5E5", edgecolor="#E5E5E5")

    # Min e Max Global
    for i in range(0, len(date)):
      if low[i] == np.min(low):
        ax.annotate('Mínimo', xy = (ax.get_xticks()[i], low[i]), xytext=(15, -15), fontsize=26,
                    textcoords='offset points', arrowprops=dict(arrowstyle='-|>'))
      if high[i] == np.max(high):
        ax.annotate('Máximo', xy = (ax.get_xticks()[i], high[i]), xytext=(15, 15), fontsize=26,
                    textcoords='offset points', arrowprops=dict(arrowstyle='-|>'))

    # Valor Atual
    ax.axhline(y = close[len(close)-1], color = "grey", linestyle='--') # Não conseguimos ler o último valor com -1

    ## Personalizando o gráfico
    # Ajustando os ticks dos eixos x e y
    plt.setp(ax, xticks = ax.get_xticks(), yticks = ax.get_yticks(),
              xticklabels = [date[i].strftime('%b %Y') for i in ax.get_xticks()],
              yticklabels= [f'R$ {valor:.2f}' for valor in ax.get_yticks()])

    # Ajustando tamanhos dos labels, retirando títulos e bordas
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='both', labelsize=32)
    ax.xaxis.set_major_locator(mticker.MaxNLocator(4))
    sns.despine()
    plt.grid(alpha=0.2)

    # Ajustando o limite de y para um respiro
    plt.ylim(ax.get_ylim()[0]-0.5, ax.get_ylim()[1]+0.5)

cotacoes["Date"] = pd.to_datetime(cotacoes["Date"], format="%Y-%m-%dT%H:%M:%S")

candlestick( cotacoes["Date"], cotacoes["Open"], cotacoes["High"], cotacoes["Low"], cotacoes["Close"])

plt.subplots_adjust(left=0.07, bottom=0.05, right=0.95, top=0.95)
plt.show()

# Dados fundamentalistas

In [11]:
# Importando as bibliotecas
import fundamentus
import pandas as pd

# Definindo a carteira de ações
carteira_fund = ["ABEV3", "B3SA3", "ELET3", "GGBR4", "ITSA4",
                 "PETR4", "RENT3", "SUZB3", "VALE3", "WEGE3"]

# Criando um df com algumas infos da carteira
ind = fundamentus.get_papel(carteira_fund)[['Setor', 'Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado',
                                            'Nro_Acoes', 'Patrim_Liq','Receita_Liquida_12m','Receita_Liquida_3m',
                                            'Lucro_Liquido_12m', 'Lucro_Liquido_3m']]

ind = ind.reset_index()
ind.rename(columns={'index':'Ativo'}, inplace=True)

colunas = ['Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado', 'Nro_Acoes', 'Patrim_Liq',
           'Receita_Liquida_12m', 'Receita_Liquida_3m', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']

ind[colunas] = ind[colunas].apply(pd.to_numeric, errors='coerce', axis=1)

ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")

# Criando um novo df com alguns indicadores da carteira
ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")

ind_2 = ind_2[['papel','P/L', 'Div.Yield','P/VP','ROE']].reset_index(drop=True)

ind_2.rename(columns={'papel': 'Ativo', 'Div.Yield':'DY'}, inplace= True)

# Concatenando os dfs em um só com as infos e indicadores
indicadores = pd.merge(ind, ind_2, on="Ativo")

# Criando uma coluna para LPA (Lucro por Ação) e VPA (Valor Patrimonial por ação)
# para calcular a fórmula de Graham  Valor intrínseco de uma ação (VI = √22,5 x LPA x VPA)
indicadores["LPA"] = (indicadores["Lucro_Liquido_12m"] / indicadores["Nro_Acoes"]).round(2)
indicadores["VPA"] = (indicadores["Patrim_Liq"] / indicadores["Nro_Acoes"]).round(2)

del ind, ind_2, colunas, carteira_fund

indicadores

,Ativo,Setor,Cotacao,Min_52_sem,Max_52_sem,Valor_de_mercado,Nro_Acoes,Patrim_Liq,Receita_Liquida_12m,Receita_Liquida_3m,Lucro_Liquido_12m,Lucro_Liquido_3m,P/L,DY,P/VP,ROE,LPA,VPA
0,ABEV3,Bebidas,12.33,11.77,14.89,1.942450e+11,1.575380e+10,7.896930e+10,7.973690e+10,1.998920e+10,1.450190e+10,4.387660e+09,13.39,0.0592,2.46,0.1836,0.92,5.01
1,B3SA3,Serviços Financeiros Diversos,11.06,10.79,14.82,6.245030e+10,5.646500e+09,2.027350e+10,9.921250e+09,2.493570e+09,4.131930e+09,9.155430e+08,15.11,0.0412,3.08,0.2038,0.73,3.59
2,ELET3,Energia Elétrica,38.38,32.78,43.99,8.854650e+10,2.307100e+09,1.123330e+11,3.715890e+10,9.922110e+09,4.549770e+09,8.920320e+08,19.46,0.0000,0.79,0.0405,1.97,48.69
3,GGBR4,Siderurgia e Metalurgia,18.63,17.06,23.40,3.928100e+10,2.108480e+09,5.191940e+10,6.625440e+10,1.621030e+10,6.339430e+09,2.043780e+09,6.20,0.0675,0.76,0.1221,3.01,24.62
4,ITSA4,Intermediários Financeiros,9.71,7.33,10.67,1.002860e+11,1.032810e+10,8.295200e+10,7.383000e+09,1.948000e+09,1.346600e+10,2.984000e+09,7.45,0.0880,1.21,0.1623,1.30,8.03
5,PETR4,"Petróleo, Gás e Biocombustíveis",42.18,19.34,42.18,5.502170e+11,1.304450e+10,3.804410e+11,5.119940e+11,1.342580e+11,1.246060e+11,3.104300e+10,4.42,0.1040,1.45,0.3275,9.55,29.16
6,RENT3,Diversos,48.51,48.51,68.96,5.199740e+10,1.071890e+09,2.538890e+10,2.890230e+10,7.908500e+09,1.805570e+09,7.058020e+08,28.80,0.0311,2.05,0.0711,1.68,23.69
7,SUZB3,Madeira e Papel,59.01,38.83,65.23,7.695600e+10,1.304120e+09,4.469280e+10,3.975560e+10,1.037150e+10,1.408480e+10,4.507340e+09,5.46,0.0200,1.72,0.3151,10.80,34.27
8,VALE3,Mineração,63.92,56.77,74.08,2.901330e+11,4.539010e+09,1.872110e+11,2.061160e+11,4.189100e+10,3.871000e+10,8.291000e+09,7.50,0.1093,1.55,0.2068,8.53,41.24
9,WEGE3,Máquinas e Equipamentos,38.85,31.12,41.02,1.630660e+11,4.197320e+09,1.740730e+10,3.284070e+10,8.033300e+09,5.752980e+09,1.327960e+09,28.34,0.0178,9.37,0.3305,1.37,4.15
